In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Load relevant tables

In [2]:
# VCT 2021
matches_overview_2021 = pd.read_csv("vct_data/vct_2021/matches/overview.csv")
matches_maps_scores_2021 = pd.read_csv("vct_data/vct_2021/matches/maps_scores.csv")
matches_eco_rounds_2021 = pd.read_csv("vct_data/vct_2021/matches/eco_rounds.csv")
matches_kills_stats_2021 = pd.read_csv("vct_data/vct_2021/matches/kills_stats.csv")
matches_win_loss_methods_count_2021 = pd.read_csv("vct_data/vct_2021/matches/win_loss_methods_count.csv")

# VCT 2022
matches_overview_2022 = pd.read_csv("vct_data/vct_2022/matches/overview.csv")
matches_maps_scores_2022 = pd.read_csv("vct_data/vct_2022/matches/maps_scores.csv")
matches_eco_rounds_2022 = pd.read_csv("vct_data/vct_2022/matches/eco_rounds.csv")
matches_kills_stats_2022 = pd.read_csv("vct_data/vct_2022/matches/kills_stats.csv")
matches_win_loss_methods_count_2022 = pd.read_csv("vct_data/vct_2022/matches/win_loss_methods_count.csv")

# VCT 2023
matches_overview_2023 = pd.read_csv("vct_data/vct_2023/matches/overview.csv")
matches_maps_scores_2023 = pd.read_csv("vct_data/vct_2023/matches/maps_scores.csv")
matches_eco_rounds_2023 = pd.read_csv("vct_data/vct_2023/matches/eco_rounds.csv")
matches_kills_stats_2023 = pd.read_csv("vct_data/vct_2023/matches/kills_stats.csv")
matches_win_loss_methods_count_2023 = pd.read_csv("vct_data/vct_2023/matches/win_loss_methods_count.csv")

# VCT 2024
matches_overview_2024 = pd.read_csv("vct_data/vct_2024/matches/overview.csv")
matches_maps_scores_2024 = pd.read_csv("vct_data/vct_2024/matches/maps_scores.csv")
matches_eco_rounds_2024 = pd.read_csv("vct_data/vct_2024/matches/eco_rounds.csv")
matches_kills_stats_2024 = pd.read_csv("vct_data/vct_2024/matches/kills_stats.csv")
matches_win_loss_methods_count_2024 = pd.read_csv("vct_data/vct_2024/matches/win_loss_methods_count.csv")

C:\Users\Red\AppData\Local\Temp\ipykernel_1924\2025941948.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_overview_2021 = pd.read_csv("vct_data/vct_2021/matches/overview.csv")


# Merge tables by columns (per year)

In [3]:
def merge_tables(matches_maps_scores, matches_overview, matches_eco_rounds, matches_kills_stats, matches_win_loss_methods_count):
    # Base table
    vct_data = matches_maps_scores.copy()

    # Tournament is sorted in descending order, reverse it to ascending order (oldest to newest)
    tourna_group = vct_data.groupby("Tournament", sort=False)
    reversed_tourna = list(tourna_group.groups.keys())[::-1]
    vct_data = pd.concat([tourna_group.get_group(tourna) for tourna in reversed_tourna]).reset_index(drop=True)

    # Aggregate and merge overview stats
    matches_overview_both = matches_overview[matches_overview["Side"] == "both"]

    overview_agg = matches_overview_both.groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]).agg({
        "Rating": "mean",
        "Average Combat Score": "mean",
        "Kills": "sum",
        "Deaths": "sum",
        "Assists": "sum",
        "Kills - Deaths (KD)": "sum",
        "Kill, Assist, Trade, Survive %": lambda x: np.mean([(float(i.strip("%")) / 100) if isinstance(i, str) else i for i in x]),
        "Average Damage Per Round": "mean",
        "Headshot %": lambda x: np.mean([(float(i.strip("%")) / 100) if isinstance(i, str) else i for i in x]),
        "First Kills": "sum",
        "First Deaths": "sum",
        "Kills - Deaths (FKD)": "sum"
    }).reset_index()

    vct_data = vct_data.merge(
        overview_agg, 
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left"
    )

    vct_data = vct_data.merge(
        overview_agg, 
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team B"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left",
        suffixes=("_TeamA", "_TeamB")
    )

    # Always drop these columns after merging for Team B to avoid duplicate col error
    vct_data = vct_data.drop(columns=["Team_TeamA", "Team_TeamB"])

    # Aggregate and merge for eco rounds
    eco_rounds_agg = matches_eco_rounds.groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]).agg({
        "Loadout Value": lambda x: np.mean([float(i.replace('k', '').replace(',', '')) * 1000 for i in x]),
        "Remaining Credits": lambda x: np.mean([float(i.replace('k', '').replace(',', '')) * 1000 for i in x]),
        "Type": lambda x: x.mode()[0]
    }).reset_index()

    vct_data = vct_data.merge(
        eco_rounds_agg,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left"
    )

    vct_data = vct_data.merge(
        eco_rounds_agg,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team B"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left",
        suffixes=("_TeamA", "_TeamB")
    )

    vct_data = vct_data.drop(columns=["Team_TeamA", "Team_TeamB"])

    # Aggregate and merge for kills stats
    kills_stats_agg = matches_kills_stats.groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]).agg({
        "2k": "sum",
        "3k": "sum",
        "4k": "sum",
        "5k": "sum",
        "1v1": "sum",
        "1v2": "sum",
        "1v3": "sum",
        "1v4": "sum",
        "1v5": "sum",
        "Econ": "mean",
        "Spike Plants": "sum",
        "Spike Defuses": "sum"
    }).reset_index()

    vct_data = vct_data.merge(
        kills_stats_agg,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left"
    )

    vct_data = vct_data.merge(
        kills_stats_agg,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team B"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left",
        suffixes=("_TeamA", "_TeamB")
    )

    vct_data = vct_data.drop(columns=["Team_TeamA", "Team_TeamB"])
    
    # Merge with win/loss methods count
    vct_data = vct_data.merge(
        matches_win_loss_methods_count,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left"
    )

    vct_data = vct_data.merge(
        matches_win_loss_methods_count,
        left_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team B"],
        right_on=["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"],
        how="left",
        suffixes=("_TeamA", "_TeamB")
    )

    vct_data = vct_data.drop(columns=["Team_TeamA", "Team_TeamB"])

    # Add prediction target
    vct_data["Winner"] = np.where(vct_data["Team A Score"] > vct_data["Team B Score"], 1, 0)

    # Remove duplicates
    vct_data.drop_duplicates(inplace=True)

    return vct_data


# Merge tables for each year
vct_2021 = merge_tables(
    matches_maps_scores_2021, 
    matches_overview_2021, 
    matches_eco_rounds_2021, 
    matches_kills_stats_2021, 
    matches_win_loss_methods_count_2021,
)

vct_2022 = merge_tables(
    matches_maps_scores_2022, 
    matches_overview_2022, 
    matches_eco_rounds_2022, 
    matches_kills_stats_2022, 
    matches_win_loss_methods_count_2022,
)

vct_2023 = merge_tables(
    matches_maps_scores_2023, 
    matches_overview_2023, 
    matches_eco_rounds_2023, 
    matches_kills_stats_2023, 
    matches_win_loss_methods_count_2023,
)

vct_2024 = merge_tables(
    matches_maps_scores_2024, 
    matches_overview_2024, 
    matches_eco_rounds_2024, 
    matches_kills_stats_2024, 
    matches_win_loss_methods_count_2024,
)

# Investigate each year's VCT data

In [ ]:
vct_2021.tail()

In [ ]:
vct_2022.tail()

In [ ]:
vct_2023.head()

In [ ]:
vct_2024.head()

# Flip Team A and Team B stats 

In [5]:
def flip_teams(df):
    df_flipped = df.copy()
    
    # Flip the columns for Team A and Team B
    df_flipped = df_flipped.rename(columns={
        'Team A': 'Team B',
        'Team A Score': 'Team B Score',
        'Team A Attacker Score': 'Team B Attacker Score',
        'Team A Defender Score': 'Team B Defender Score',
        'Team A Overtime Score': 'Team B Overtime Score',
        'Team B': 'Team A',
        'Team B Score': 'Team A Score',
        'Team B Attacker Score': 'Team A Attacker Score',
        'Team B Defender Score': 'Team A Defender Score',
        'Team B Overtime Score': 'Team A Overtime Score',
        'Rating_TeamA': 'Rating_TeamB',
        'Average Combat Score_TeamA': 'Average Combat Score_TeamB',
        'Kills_TeamA': 'Kills_TeamB',
        'Deaths_TeamA': 'Deaths_TeamB',
        'Assists_TeamA': 'Assists_TeamB',
        'Kills - Deaths (KD)_TeamA': 'Kills - Deaths (KD)_TeamB',
        'Kill, Assist, Trade, Survive %_TeamA': 'Kill, Assist, Trade, Survive %_TeamB',
        'Average Damage Per Round_TeamA': 'Average Damage Per Round_TeamB',
        'Headshot %_TeamA': 'Headshot %_TeamB',
        'First Kills_TeamA': 'First Kills_TeamB',
        'First Deaths_TeamA': 'First Deaths_TeamB',
        'Kills - Deaths (FKD)_TeamA': 'Kills - Deaths (FKD)_TeamB',
        'Rating_TeamB': 'Rating_TeamA',
        'Average Combat Score_TeamB': 'Average Combat Score_TeamA',
        'Kills_TeamB': 'Kills_TeamA',
        'Deaths_TeamB': 'Deaths_TeamA',
        'Assists_TeamB': 'Assists_TeamA',
        'Kills - Deaths (KD)_TeamB': 'Kills - Deaths (KD)_TeamA',
        'Kill, Assist, Trade, Survive %_TeamB': 'Kill, Assist, Trade, Survive %_TeamA',
        'Average Damage Per Round_TeamB': 'Average Damage Per Round_TeamA',
        'Headshot %_TeamB': 'Headshot %_TeamA',
        'First Kills_TeamB': 'First Kills_TeamA',
        'First Deaths_TeamB': 'First Deaths_TeamA',
        'Kills - Deaths (FKD)_TeamB': 'Kills - Deaths (FKD)_TeamA',
        'Loadout Value_TeamA': 'Loadout Value_TeamB',
        'Remaining Credits_TeamA': 'Remaining Credits_TeamB',
        'Type_TeamA': 'Type_TeamB',
        'Loadout Value_TeamB': 'Loadout Value_TeamA',
        'Remaining Credits_TeamB': 'Remaining Credits_TeamA',
        'Type_TeamB': 'Type_TeamA',
        '2k_TeamA': '2k_TeamB',
        '3k_TeamA': '3k_TeamB',
        '4k_TeamA': '4k_TeamB',
        '5k_TeamA': '5k_TeamB',
        '1v1_TeamA': '1v1_TeamB',
        '1v2_TeamA': '1v2_TeamB',
        '1v3_TeamA': '1v3_TeamB',
        '1v4_TeamA': '1v4_TeamB',
        '1v5_TeamA': '1v5_TeamB',
        'Econ_TeamA': 'Econ_TeamB',
        'Spike Plants_TeamA': 'Spike Plants_TeamB',
        'Spike Defuses_TeamA': 'Spike Defuses_TeamB',
        '2k_TeamB': '2k_TeamA',
        '3k_TeamB': '3k_TeamA',
        '4k_TeamB': '4k_TeamA',
        '5k_TeamB': '5k_TeamA',
        '1v1_TeamB': '1v1_TeamA',
        '1v2_TeamB': '1v2_TeamA',
        '1v3_TeamB': '1v3_TeamA',
        '1v4_TeamB': '1v4_TeamA',
        '1v5_TeamB': '1v5_TeamA',
        'Econ_TeamB': 'Econ_TeamA',
        'Spike Plants_TeamB': 'Spike Plants_TeamA',
        'Spike Defuses_TeamB': 'Spike Defuses_TeamA',
        'Elimination_TeamA': 'Elimination_TeamB',
        'Detonated_TeamA': 'Detonated_TeamB',
        'Defused_TeamA': 'Defused_TeamB',
        'Time Expiry (No Plant)_TeamA': 'Time Expiry (No Plant)_TeamB',
        'Eliminated_TeamA': 'Eliminated_TeamB',
        'Defused Failed_TeamA': 'Defused Failed_TeamB',
        'Detonation Denied_TeamA': 'Detonation Denied_TeamB',
        'Time Expiry (Failed to Plant)_TeamA': 'Time Expiry (Failed to Plant)_TeamB',
        'Elimination_TeamB': 'Elimination_TeamA',
        'Detonated_TeamB': 'Detonated_TeamA',
        'Defused_TeamB': 'Defused_TeamA',
        'Time Expiry (No Plant)_TeamB': 'Time Expiry (No Plant)_TeamA',
        'Eliminated_TeamB': 'Eliminated_TeamA',
        'Defused Failed_TeamB': 'Defused Failed_TeamA',
        'Detonation Denied_TeamB': 'Detonation Denied_TeamA',
        'Time Expiry (Failed to Plant)_TeamB': 'Time Expiry (Failed to Plant)_TeamA'
    })

    # Flip the winner column
    df_flipped['Winner'] = df_flipped['Winner'].apply(lambda x: 1 if x == 0 else 0)

    original_order = [
        'Tournament',
        'Stage',
        'Match Type',
        'Match Name',
        'Map',
        'Team A',
        'Team A Score',
        'Team A Attacker Score',
        'Team A Defender Score',
        'Team A Overtime Score',
        'Team B',
        'Team B Score',
        'Team B Attacker Score',
        'Team B Defender Score',
        'Team B Overtime Score',
        'Duration',
        'Rating_TeamA',
        'Average Combat Score_TeamA',
        'Kills_TeamA',
        'Deaths_TeamA',
        'Assists_TeamA',
        'Kills - Deaths (KD)_TeamA',
        'Kill, Assist, Trade, Survive %_TeamA',
        'Average Damage Per Round_TeamA',
        'Headshot %_TeamA',
        'First Kills_TeamA',
        'First Deaths_TeamA',
        'Kills - Deaths (FKD)_TeamA',
        'Rating_TeamB',
        'Average Combat Score_TeamB',
        'Kills_TeamB',
        'Deaths_TeamB',
        'Assists_TeamB',
        'Kills - Deaths (KD)_TeamB',
        'Kill, Assist, Trade, Survive %_TeamB',
        'Average Damage Per Round_TeamB',
        'Headshot %_TeamB',
        'First Kills_TeamB',
        'First Deaths_TeamB',
        'Kills - Deaths (FKD)_TeamB',
        'Loadout Value_TeamA',
        'Remaining Credits_TeamA',
        'Type_TeamA',
        'Loadout Value_TeamB',
        'Remaining Credits_TeamB',
        'Type_TeamB',
        '2k_TeamA',
        '3k_TeamA',
        '4k_TeamA',
        '5k_TeamA',
        '1v1_TeamA',
        '1v2_TeamA',
        '1v3_TeamA',
        '1v4_TeamA',
        '1v5_TeamA',
        'Econ_TeamA',
        'Spike Plants_TeamA',
        'Spike Defuses_TeamA',
        '2k_TeamB',
        '3k_TeamB',
        '4k_TeamB',
        '5k_TeamB',
        '1v1_TeamB',
        '1v2_TeamB',
        '1v3_TeamB',
        '1v4_TeamB',
        '1v5_TeamB',
        'Econ_TeamB',
        'Spike Plants_TeamB',
        'Spike Defuses_TeamB',
        'Elimination_TeamA',
        'Detonated_TeamA',
        'Defused_TeamA',
        'Time Expiry (No Plant)_TeamA',
        'Eliminated_TeamA',
        'Defused Failed_TeamA',
        'Detonation Denied_TeamA',
        'Time Expiry (Failed to Plant)_TeamA',
        'Elimination_TeamB',
        'Detonated_TeamB',
        'Defused_TeamB',
        'Time Expiry (No Plant)_TeamB',
        'Eliminated_TeamB',
        'Defused Failed_TeamB',
        'Detonation Denied_TeamB',
        'Time Expiry (Failed to Plant)_TeamB',
        'Winner'
    ]


    df_flipped = df_flipped[original_order]

    return df_flipped


# Apply to all years
vct_2021_flipped = flip_teams(vct_2021)
vct_2022_flipped = flip_teams(vct_2022)
vct_2023_flipped = flip_teams(vct_2023)
vct_2024_flipped = flip_teams(vct_2024)

Checking if the flip worked

In [6]:
vct_2024.tail()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
1099,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Haven,EDward Gaming,6,3,3,NaN,Team Heretics,13,4,9,NaN,1:16:43,0.814,182.0,59.0,73.0,21.0,-14.0,0.672,118.0,0.272,8.0,11.0,-3.0,1.208,219.8,73.0,59.0,38.0,14.0,0.802,146.2,0.368,11.0,8.0,3.0,17389.473684,6926.315789,Full buy: 20k+,18194.736842,9878.947368,Full buy: 20k+,11.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,43.8,9.0,0.0,19.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,62.6,3.0,3.0,5,1,0,0,7,0,3,3,7,0,3,3,5,1,0,0,0
1100,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Sunset,EDward Gaming,13,11,2,NaN,Team Heretics,4,3,1,NaN,45:11,1.364,225.6,70.0,40.0,37.0,30.0,0.868,144.8,0.370,12.0,5.0,7.0,0.642,142.0,40.0,70.0,12.0,-30.0,0.542,97.0,0.306,5.0,12.0,-7.0,18494.117647,18241.176471,Full buy: 20k+,15641.176471,4694.117647,Full buy: 20k+,14.0,0.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,74.2,12.0,1.0,9.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,41.8,4.0,1.0,9,3,1,0,3,0,1,0,3,0,1,0,9,3,1,0,1
1101,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Lotus,EDward Gaming,13,6,7,NaN,Team Heretics,9,3,6,NaN,55:30,1.170,228.4,87.0,72.0,47.0,15.0,0.774,143.4,0.320,11.0,11.0,0.0,0.836,184.4,71.0,87.0,26.0,-16.0,0.608,118.0,0.366,11.0,11.0,0.0,18436.363636,8027.272727,Full buy: 20k+,16540.909091,5600.000000,Full buy: 20k+,13.0,9.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,58.6,8.0,2.0,13.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,45.8,8.0,1.0,10,0,2,1,7,1,1,0,7,1,1,0,10,0,2,1,1
1102,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Bind,EDward Gaming,11,6,5,NaN,Team Heretics,13,7,6,NaN,1:01:45,0.976,203.2,86.0,91.0,32.0,-5.0,0.782,139.8,0.304,10.0,14.0,-4.0,1.038,210.0,91.0,86.0,40.0,5.0,0.818,129.4,0.302,14.0,10.0,4.0,17150.000000,5462.500000,Full buy: 20k+,18404.166667,7033.333333,Full buy: 20k+,14.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,55.2,10.0,3.0,18.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,47.8,10.0,4.0,7,1,3,0,7,1,4,1,7,1,4,1,7,1,3,0,0
1103,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Abyss,EDward Gaming,13,8,5,NaN,Team Heretics,9,5,4,NaN,51:55,1.194,208.6,84.0,63.0,36.0,21.0,0.800,140.0,0.310,13.0,9.0,4.0,0.772,175.4,63.0,88.0,25.0,-25.0,0.692,115.8,0.358,9.0,13.0,-4.0,18486.363636,12090.909091,Full buy: 20k+,15363.636364,8759.090909,Full buy: 20k+,14.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,61.8,12.0,2.0,11.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,48.0,7.0,2.0,11,0,2,0,2,3,2,2,2,3,2,2,11,0,2,0,1


In [7]:
vct_2024_flipped.tail()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
1099,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Haven,Team Heretics,13,4,9,NaN,EDward Gaming,6,3,3,NaN,1:16:43,1.208,219.8,73.0,59.0,38.0,14.0,0.802,146.2,0.368,11.0,8.0,3.0,0.814,182.0,59.0,73.0,21.0,-14.0,0.672,118.0,0.272,8.0,11.0,-3.0,18194.736842,9878.947368,Full buy: 20k+,17389.473684,6926.315789,Full buy: 20k+,19.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,62.6,3.0,3.0,11.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,43.8,9.0,0.0,7,0,3,3,5,1,0,0,5,1,0,0,7,0,3,3,1
1100,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Sunset,Team Heretics,4,3,1,NaN,EDward Gaming,13,11,2,NaN,45:11,0.642,142.0,40.0,70.0,12.0,-30.0,0.542,97.0,0.306,5.0,12.0,-7.0,1.364,225.6,70.0,40.0,37.0,30.0,0.868,144.8,0.370,12.0,5.0,7.0,15641.176471,4694.117647,Full buy: 20k+,18494.117647,18241.176471,Full buy: 20k+,9.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,41.8,4.0,1.0,14.0,0.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,74.2,12.0,1.0,3,0,1,0,9,3,1,0,9,3,1,0,3,0,1,0,0
1101,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Lotus,Team Heretics,9,3,6,NaN,EDward Gaming,13,6,7,NaN,55:30,0.836,184.4,71.0,87.0,26.0,-16.0,0.608,118.0,0.366,11.0,11.0,0.0,1.170,228.4,87.0,72.0,47.0,15.0,0.774,143.4,0.320,11.0,11.0,0.0,16540.909091,5600.000000,Full buy: 20k+,18436.363636,8027.272727,Full buy: 20k+,13.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,45.8,8.0,1.0,13.0,9.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,58.6,8.0,2.0,7,1,1,0,10,0,2,1,10,0,2,1,7,1,1,0,0
1102,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Bind,Team Heretics,13,7,6,NaN,EDward Gaming,11,6,5,NaN,1:01:45,1.038,210.0,91.0,86.0,40.0,5.0,0.818,129.4,0.302,14.0,10.0,4.0,0.976,203.2,86.0,91.0,32.0,-5.0,0.782,139.8,0.304,10.0,14.0,-4.0,18404.166667,7033.333333,Full buy: 20k+,17150.000000,5462.500000,Full buy: 20k+,18.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,47.8,10.0,4.0,14.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,55.2,10.0,3.0,7,1,4,1,7,1,3,0,7,1,3,0,7,1,4,1,1
1103,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Abyss,Team Heretics,9,5,4,NaN,EDward Gaming,13,8,5,NaN,51:55,0.772,175.4,63.0,88.0,25.0,-25.0,0.692,115.8,0.358,9.0,13.0,-4.0,1.194,208.6,84.0,63.0,36.0,21.0,0.800,140.0,0.310,13.0,9.0,4.0,15363.636364,8759.090909,Full buy: 20k+,18486.363636,12090.909091,Full buy: 20k+,11.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,48.0,7.0,2.0,14.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,61.8,12.0,2.0,2,3,2,2,11,0,2,0,11,0,2,0,2,3,2,2,0


# Combine the two variants together

In [9]:
def alternate_concat(df, df_flipped):
    if len(df) != len(df_flipped):
        raise ValueError("Both DataFrames must have the same number of rows")
    
    # Convert to NumPy arrays for faster processing
    arr1 = df.to_numpy()
    arr2 = df_flipped.to_numpy()
    
    # Initialize result array with the correct shape
    result = np.empty((len(df) * 2, df.shape[1]), dtype=arr1.dtype)
    
    # Even indices (0, 2, 4, ...) get rows from df
    result[0::2] = arr1

    # Odd indices (1, 3, 5, ...) get rows from df_flipped
    result[1::2] = arr2
    
    # Convert back to DataFrame
    return pd.DataFrame(result, columns=df.columns)

# Concatenate the original and flipped DataFrames for each year
vct_2021_combined = alternate_concat(vct_2021, vct_2021_flipped)
vct_2022_combined = alternate_concat(vct_2022, vct_2022_flipped)
vct_2023_combined = alternate_concat(vct_2023, vct_2023_flipped)
vct_2024_combined = alternate_concat(vct_2024, vct_2024_flipped)

In [10]:
vct_2024_combined.tail()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Rating_TeamA,Average Combat Score_TeamA,Kills_TeamA,Deaths_TeamA,Assists_TeamA,Kills - Deaths (KD)_TeamA,"Kill, Assist, Trade, Survive %_TeamA",Average Damage Per Round_TeamA,Headshot %_TeamA,First Kills_TeamA,First Deaths_TeamA,Kills - Deaths (FKD)_TeamA,Rating_TeamB,Average Combat Score_TeamB,Kills_TeamB,Deaths_TeamB,Assists_TeamB,Kills - Deaths (KD)_TeamB,"Kill, Assist, Trade, Survive %_TeamB",Average Damage Per Round_TeamB,Headshot %_TeamB,First Kills_TeamB,First Deaths_TeamB,Kills - Deaths (FKD)_TeamB,Loadout Value_TeamA,Remaining Credits_TeamA,Type_TeamA,Loadout Value_TeamB,Remaining Credits_TeamB,Type_TeamB,2k_TeamA,3k_TeamA,4k_TeamA,5k_TeamA,1v1_TeamA,1v2_TeamA,1v3_TeamA,1v4_TeamA,1v5_TeamA,Econ_TeamA,Spike Plants_TeamA,Spike Defuses_TeamA,2k_TeamB,3k_TeamB,4k_TeamB,5k_TeamB,1v1_TeamB,1v2_TeamB,1v3_TeamB,1v4_TeamB,1v5_TeamB,Econ_TeamB,Spike Plants_TeamB,Spike Defuses_TeamB,Elimination_TeamA,Detonated_TeamA,Defused_TeamA,Time Expiry (No Plant)_TeamA,Eliminated_TeamA,Defused Failed_TeamA,Detonation Denied_TeamA,Time Expiry (Failed to Plant)_TeamA,Elimination_TeamB,Detonated_TeamB,Defused_TeamB,Time Expiry (No Plant)_TeamB,Eliminated_TeamB,Defused Failed_TeamB,Detonation Denied_TeamB,Time Expiry (Failed to Plant)_TeamB,Winner
2203,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Lotus,Team Heretics,9,3,6,NaN,EDward Gaming,13,6,7,NaN,55:30,0.836,184.4,71.0,87.0,26.0,-16.0,0.608,118.0,0.366,11.0,11.0,0.0,1.17,228.4,87.0,72.0,47.0,15.0,0.774,143.4,0.32,11.0,11.0,0.0,16540.909091,5600.0,Full buy: 20k+,18436.363636,8027.272727,Full buy: 20k+,13.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,45.8,8.0,1.0,13.0,9.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,58.6,8.0,2.0,7,1,1,0,10,0,2,1,10,0,2,1,7,1,1,0,0
2204,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Bind,EDward Gaming,11,6,5,NaN,Team Heretics,13,7,6,NaN,1:01:45,0.976,203.2,86.0,91.0,32.0,-5.0,0.782,139.8,0.304,10.0,14.0,-4.0,1.038,210.0,91.0,86.0,40.0,5.0,0.818,129.4,0.302,14.0,10.0,4.0,17150.0,5462.5,Full buy: 20k+,18404.166667,7033.333333,Full buy: 20k+,14.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,55.2,10.0,3.0,18.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,47.8,10.0,4.0,7,1,3,0,7,1,4,1,7,1,4,1,7,1,3,0,0
2205,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Bind,Team Heretics,13,7,6,NaN,EDward Gaming,11,6,5,NaN,1:01:45,1.038,210.0,91.0,86.0,40.0,5.0,0.818,129.4,0.302,14.0,10.0,4.0,0.976,203.2,86.0,91.0,32.0,-5.0,0.782,139.8,0.304,10.0,14.0,-4.0,18404.166667,7033.333333,Full buy: 20k+,17150.0,5462.5,Full buy: 20k+,18.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,47.8,10.0,4.0,14.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,55.2,10.0,3.0,7,1,4,1,7,1,3,0,7,1,3,0,7,1,4,1,1
2206,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Abyss,EDward Gaming,13,8,5,NaN,Team Heretics,9,5,4,NaN,51:55,1.194,208.6,84.0,63.0,36.0,21.0,0.8,140.0,0.31,13.0,9.0,4.0,0.772,175.4,63.0,88.0,25.0,-25.0,0.692,115.8,0.358,9.0,13.0,-4.0,18486.363636,12090.909091,Full buy: 20k+,15363.636364,8759.090909,Full buy: 20k+,14.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,61.8,12.0,2.0,11.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,48.0,7.0,2.0,11,0,2,0,2,3,2,2,2,3,2,2,11,0,2,0,1
2207,Valorant Champions 2024,Playoffs,Grand Final,EDward Gaming vs Team Heretics,Abyss,Team Heretics,9,5,4,NaN,EDward Gaming,13,8,5,NaN,51:55,0.772,175.4,63.0,88.0,25.0,-25.0,0.692,115.8,0.358,9.0,13.0,-4.0,1.194,208.6,84.0,63.0,36.0,21.0,0.8,140.0,0.31,13.0,9.0,4.0,15363.636364,8759.090909,Full buy: 20k+,18486.363636,12090.909091,Full buy: 20k+,11.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,48.0,7.0,2.0,14.0,5.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,61.8,12.0,2.0,2,3,2,2,11,0,2,0,11,0,2,0,2,3,2,2,0


# Allocate yearly data into train and test sets

Use 2021-2023 data for training while 2024 data for testing

In [11]:
train_data = pd.concat([vct_2021_combined, vct_2022_combined, vct_2023_combined], ignore_index=True)
test_data = vct_2024_combined

train_data.to_csv("vct_data/train_raw.csv", index=False)
test_data.to_csv("vct_data/test_raw.csv", index=False)